In [54]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, LpAffineExpression


In [14]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMinimize)

In [15]:
p1 = LpVariable(name="p1", lowBound=0, upBound=1)
p21 = LpVariable(name="p21", lowBound=0, upBound=1)
p22 = LpVariable(name="p22", lowBound=0, upBound=1)
p31 = LpVariable(name="p31", lowBound=0, upBound=1)
p32 = LpVariable(name="p32", lowBound=0, upBound=1)
p33 = LpVariable(name="p33", lowBound=0, upBound=1)
p34 = LpVariable(name="p34", lowBound=0, upBound=1)
c = LpVariable(name="c")

In [16]:
con1 = (1-p1)+(1-p21)+(1-p31) <= c
con2 = (1-p1)+(1-p21)+(p31-1) <= c
con3 = (1-p1)+(p21-1)+(1-p32) <= c
con4 = (-p1)+(p21)+(p32) <= c
con5 = (p1-1)+(1-p22)+(1-p33) <= c
con6 = (p1)+(-p22)+(p33) <= c
con7 = (p1)+(p22)+(p34) <= c
con8 = (p1)+(p22)+(-p34) <= c


In [17]:
model += con1 
model += con2
model += con3  
model += con4
model += con5
model += con6
model += con7
model += con8


In [18]:
model += c

In [19]:
model

small-problem:
MINIMIZE
1*c + 0
SUBJECT TO
_C1: - c - p1 - p21 - p31 <= -3

_C2: - c - p1 - p21 + p31 <= -1

_C3: - c - p1 + p21 - p32 <= -1

_C4: - c - p1 + p21 + p32 <= 0

_C5: - c + p1 - p22 - p33 <= -1

_C6: - c + p1 - p22 + p33 <= 0

_C7: - c + p1 + p22 + p34 <= 0

_C8: - c + p1 + p22 - p34 <= 0

VARIABLES
c free Continuous
p1 <= 1 Continuous
p21 <= 1 Continuous
p22 <= 1 Continuous
p31 <= 1 Continuous
p32 <= 1 Continuous
p33 <= 1 Continuous
p34 <= 1 Continuous

In [20]:
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/hadar/Dev/RL/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_7/xf1wjsfx5bs9p2klwbsmtf640000gq/T/a827583d0c22419dbe020345ff55af31-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_7/xf1wjsfx5bs9p2klwbsmtf640000gq/T/a827583d0c22419dbe020345ff55af31-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 47 RHS
At line 56 BOUNDS
At line 65 ENDATA
Problem MODEL has 8 rows, 8 columns and 32 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 8 (0) rows, 8 (0) columns and 32 (0) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 0 ( 0%) - largest zero change 5.8145531e-05
0  Obj 0 Primal inf 5.9999996 (4) Dual inf 0.0099999 (1) w.o. free dual inf (0)
7  Obj 0.75004551
Optimal - objective value 0.75
Optimal objective 0.75 - 7 i

In [21]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

c: 0.75
p1: 0.5
p21: 0.75
p22: 0.25
p31: 1.0
p32: 0.5
p33: 0.5
p34: 0.0


In [60]:
class Node:
    def __init__(self, value, parent=None):
        self.value = value
        self.lp_variable = LpVariable(name=f"p{value}", lowBound=0, upBound=1)
        self.left = None
        self.right = None
        self.parent = parent
    
    def get_left_child(self):
        return self.left
    
    def get_right_child(self):
        return self.right
    
    def get_parent(self):
        return self.parent
    
    def print_tree(self, level=0):
        """Recursively prints the tree structure."""
        print(' ' * 4 * level + f'- {self.value}')
        if self.left:
            self.left.print_tree(level + 1)
        if self.right:
            self.right.print_tree(level + 1)
            

def generate_full_binary_tree(depth, current_depth=0, parent=None, value=1):
    """Recursively generates a full binary tree to the specified depth."""
    if depth < 0 or current_depth > depth:
        return None
    
    # Create the current node
    node = Node(value, parent)
    
    # If not at the desired depth, create left and right children
    if current_depth < depth:
        node.left = generate_full_binary_tree(depth, current_depth + 1, node, value*2)
        node.right = generate_full_binary_tree(depth, current_depth + 1, node, value*2 + 1)
    
    return node


def get_lp_expressions(node, expression_0=None, expression_1=None):
    if not node.left and not node.right: # Leaf node
        return [expression_0, expression_1]
    
    expression_left_0 = expression_0 + (1 - node.lp_variable)
    expression_left_1 = expression_1 + (-node.lp_variable)
    expressions_left = get_lp_expressions(node.left, expression_left_0, expression_left_1)
    
    expression_right_0 = expression_0 + (node.lp_variable - 1)
    expression_right_1 = expression_1 + (node.lp_variable)
    expressions_right = get_lp_expressions(node.right, expression_right_0, expression_right_1)
    
    #return all the results as array with 1 dimension
    return expressions_left + expressions_right
    
    

def get_lp_constraints(node):
    c = LpVariable(name="c")
    constraints_dict = {}
    
    expressions = get_lp_expressions(node, LpAffineExpression(), LpAffineExpression())
    
    for expression in expressions:
        # Separate the LHS and RHS of the expression
        lhs_expression = expression - expression.constant
        rhs_value = -expression.constant  # Assuming expression is <= 0
        
        lhs_str = str(lhs_expression)  # Use LHS for comparison
        
        # Check if this LHS is already in the dictionary
        if lhs_str in constraints_dict:
            # Keep the most restrictive constraint (smaller RHS value)
            existing_rhs_value = -constraints_dict[lhs_str].constant
            if rhs_value < existing_rhs_value:
                constraints_dict[lhs_str] = lhs_expression - c <= rhs_value
        else:
            constraints_dict[lhs_str] = lhs_expression - c <= rhs_value
    
    # Return the list of unique, most restrictive constraints
    return list(constraints_dict.values())

    
# Example usage:|
T = 3 # Depth of the tree
root = generate_full_binary_tree(T)
    
constraints = get_lp_constraints(root) 
for constrains in constraints:
    print(constrains)



-c - p1 - p2 - p4 <= -3
-c - p1 - p2 + p4 <= -1
-c - p1 + p2 - p5 <= -1
-c - p1 + p2 + p5 <= 0
-c + p1 - p3 - p6 <= -1
-c + p1 - p3 + p6 <= 0
-c + p1 + p3 - p7 <= 0
-c + p1 + p3 + p7 <= 0


In [ ]:

model2 = LpProblem(name="experts-problem-1", sense=LpMinimize)
model2 += c
for constraint in constraints:
    model2 += constraint

In [62]:
model2

experts-problem-1:
MINIMIZE
1*c + 0
SUBJECT TO
_C1: - c - p1 - p2 - p4 <= -3

_C2: - c - p1 - p2 + p4 <= -1

_C3: - c - p1 + p2 - p5 <= -1

_C4: - c - p1 + p2 + p5 <= 0

_C5: - c + p1 - p3 - p6 <= -1

_C6: - c + p1 - p3 + p6 <= 0

_C7: - c + p1 + p3 - p7 <= 0

_C8: - c + p1 + p3 + p7 <= 0

VARIABLES
c free Continuous
c free Continuous
p1 <= 1 Continuous
p2 <= 1 Continuous
p3 <= 1 Continuous
p4 <= 1 Continuous
p5 <= 1 Continuous
p6 <= 1 Continuous
p7 <= 1 Continuous

In [61]:
model2.solve()
for var in model2.variables():
    print(f"{var.name}: {var.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/hadar/Dev/RL/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_7/xf1wjsfx5bs9p2klwbsmtf640000gq/T/0298bd514b6d451f9225b65071810649-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_7/xf1wjsfx5bs9p2klwbsmtf640000gq/T/0298bd514b6d451f9225b65071810649-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
Duplicate row C0000000 at line 22 <     X0000001  C0000000  -1.000000000000e+00 >
Duplicate row C0000001 at line 23 <     X0000001  C0000001  -1.000000000000e+00 >
Duplicate row C0000002 at line 24 <     X0000001  C0000002  -1.000000000000e+00 >
Duplicate row C0000003 at line 25 <     X0000001  C0000003  -1.000000000000e+00 >
Duplicate row C0000004 at line 26 <     X0000001  C0000004  -1.000000000000e+00 >
Duplicate row C0000005 at line 27 <     X0000001  C0000005  -1.000000000000e+00 >
Duplicate

PulpSolverError: Pulp: Error while executing /Users/hadar/Dev/RL/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc

In [30]:



# These are just to show how to use the get_left_child, get_right_child, and get_parent methods.
# For a tree with depth T=2, let's access the left child of the root and its right child
left_child = root.get_left_child()
right_child_of_left_child = left_child.get_right_child()
parent_of_right_child_of_left_child = right_child_of_left_child.get_parent()

print(f"Left child of root: {left_child.value}")
print(f"Right child of left child: {right_child_of_left_child.value}")
print(f"Parent of the right child of left child: {parent_of_right_child_of_left_child.value}")



Left child of root: 2
Right child of left child: 5
Parent of the right child of left child: 2


In [31]:
root.print_tree()

- 1
    - 2
        - 4
            - 8
            - 9
        - 5
            - 10
            - 11
    - 3
        - 6
            - 12
            - 13
        - 7
            - 14
            - 15
